In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import seaborn as sns

In [ ]:
#importing training set
train_dir = pd.read_csv('../input/digit-recognizer/train.csv')
train_dir.head()

In [ ]:
train_dir.shape

In [ ]:
#defining train labels and train sets
train_label = np.array(train_dir['label'])
train_set = np.array(train_dir.drop(['label'], axis = 1))

In [ ]:
sns.countplot(train_label)

In [ ]:
#importing test set
test_set = pd.read_csv('../input/digit-recognizer/test.csv')
test_set.head()

In [ ]:
#normalizing train set and test set
train_set = train_set/255.
test_set = test_set/255.

In [ ]:
#reshaping the images to 3d matrices for the input of neural network(shape of input image 28x28)
#here 1 means images are grayscale
train_set = train_set.reshape(-1,28,28,1)

#converting test_set into an numpy array
test_set = np.array(test_set)
test_set = test_set.reshape(-1,28,28,1)

In [ ]:
#training labels are in range(0,9),hence applying onehotecoding()
from tensorflow.keras.utils import to_categorical
train_label = to_categorical(train_label,10)

In [ ]:
#classes of training set
names = ['0','1','2','3','4','5','6','7','8','9']

plt.figure(figsize=(6, 6))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_set[i])
    plt.title(names[train_label[i].argmax()])
    plt.tight_layout()
plt.show()

In [ ]:
# Split the train and the validation set for the fitting
from sklearn.model_selection import train_test_split
train_set, val_set, train_label, val_label = train_test_split(train_set, train_label, test_size = 0.1, random_state=2)

In [ ]:
#creating the model
from keras.layers import Conv2D,Dense,MaxPool2D,Flatten,Dropout

#defining the model as a sequence of layers.
model = keras.models.Sequential()

#adding 1st Convolutional layer
model.add(Conv2D(filters = 32,kernel_size = 3,activation = 'relu' ,input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=2))
model.add(Dropout(0.2))

#adding 2nd Convolutional layer
model.add(Conv2D(filters = 32,kernel_size = 3,activation = 'relu' ,input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=2))
model.add(Dropout(0.2))

#flattening the output
model.add(Flatten())

#adding dense layer
model.add(Dense(128,activation = "relu"))
model.add(Dropout(0.2))

#adding final layer
model.add(Dense(10,activation = "softmax"))

#compiling the model
model.compile(optimizer=keras.optimizers.Adam(lr=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])

#training the model
cnn = model.fit(train_set, train_label, epochs=30, batch_size = 16 ,validation_data = (val_set,val_label))

In [ ]:
#summary of model
model.summary()

In [ ]:
#evaluating model on test set
model.evaluate(val_set,val_label)

In [ ]:
#storing predictions in variable(pred)
pred=model.predict(val_set,verbose=1)

In [ ]:
#saving index of maximum value of pred in preds
preds = []
for i in range(0,len(pred)):
    preds.append(pred[i].argmax())

#saving index of maximum value of training set in actual
actual = []
for i in range(0,len(val_label)):
    actual.append(val_label[i].argmax())

In [ ]:
#classification report
from sklearn import metrics

report = metrics.classification_report(actual, preds, target_names = names)
print(report)

In [ ]:
#graph for accuracy
plt.plot(cnn.history['accuracy'])
plt.plot(cnn.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

#graph for loss
plt.plot(cnn.history['loss'])
plt.plot(cnn.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()